In [1]:
import requests
import json
import time
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [2]:
username = ""
password = ""

contest_id = "ahc001"
problem_id = "ahc001_a"

contest_start = datetime.datetime(2022,10,13,19,0,0)
contest_end = datetime.datetime(2022,10,20,19,0,0)

In [3]:
personal = {
 'netobas': '家買え',
 'take911': 'a･Cyan',
 'akane831': '家買え',
 'matsu7874': '松竹梅',
 'hiraku': 'ブレインバクハーズ',
 'nakashiii': '家買え',
 'michirakara': '家買え',
 'gengar_094': 'ブレインバクハーズ',
 'ainem': '運営',
 'keroru': '松竹梅',
 'tnodino': 'ブレインバクハーズ',
 'r3yohei': 'バベルクライマーズ',
 'negibose2020': 'ブレインバクハーズ',
 'Gex777': '家買え',
 'kaede2020': 'a･Cyan',
 'jabee': 'a･Cyan',
 'meowmeowcats': 'バベルクライマーズ',
 'fujikawahiroaki': 'バベルクライマーズ',
 'jaku': 'a･Cyan',
 'takekinmkk': '松竹梅',
 'ponjuice': '松竹梅'
}

In [4]:
df=pd.DataFrame.from_dict(personal, orient="index", columns=["チーム名"])
df.to_csv("personal_info.csv")

In [5]:
personal = pd.read_csv("personal_info.csv", index_col=0)
personal.head()

,チーム名
netobas,家買え
take911,a･Cyan
akane831,家買え
matsu7874,松竹梅
hiraku,ブレインバクハーズ


In [6]:
# User Agentの設定の設定
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4.1 Safari/605.1.15"
header = {
    'User-Agent': user_agent
}

login_url = "https://atcoder.jp/login?continue=https%3A%2F%2Fatcoder.jp%2F"
session = requests.session()
response = session.get(login_url, headers=header)

# BeautifulSoupオブジェクト作成(token取得の為)
bs = BeautifulSoup(response.text, 'html.parser')
# tokenの取得
csrf_token = bs.find(attrs={'name':'csrf_token'}).get('value')

login_data = {
   'username': username,
   'password': password,
   'csrf_token': csrf_token,
}

# cookieの取得
response_cookie = response.cookies

login = session.post(login_url, data=login_data, cookies=response_cookie)
time.sleep(2) # 少し時間を置いてみる

In [7]:
standings_url = "https://atcoder.jp/contests/" + contest_id + "/standings/json"
standings_json = session.get(standings_url).json()

In [8]:
def make_datetime(s: str) -> datetime:
  ymd, hms = s.split(" ")
  year, month, day = map(int,ymd.split("-"))
  hour, minute, second = hms.split(":")
  second = int(second.split("+")[0])

  res = datetime.datetime(year=year, month=month, day=day, hour=int(hour), minute=int(minute), second=int(second))

  return res

In [9]:
records = {
    "name": [],
    "team": [],
    "lang": [],
    "score": [],
    "submit": [],
    "elapsed_time": [],
    #"argo_rate": [],
    #"heuristic_rate": [],
    #"performance": []
}

submission_url = "https://atcoder.jp/contests/" + contest_id +"/submissions?desc=true&f.LanguageName=&f.Status=AC&f.Task=" + problem_id + "&f.User=&orderBy=created&page="
page=1

not_in_record = set()

while True:
  submissions = session.get(submission_url + str(page))
  soup = BeautifulSoup(submissions.text, "html5lib")
  tr = soup.select("tbody > tr")
  if len(tr)==0:
    break
  for row in tr:
    submit = make_datetime(row.contents[1].text)
    if contest_start < submit < contest_end:
      name = row.contents[5].text.rstrip() if " " in row.contents[5].text else row.contents[5].text
      lang = row.contents[7].text
      score = int(row.find(name="td", attrs={"class": "submission-score"}).text)
      elapsed_time = row.contents[14].text
      if name in personal.index:
        tname = personal.at[name, "チーム名"]
        records["name"].append(name)
        records["team"].append(tname)
        records["lang"].append(lang)
        records["score"].append(score)
        records["submit"].append(submit)
        records["elapsed_time"].append(elapsed_time)
        #records["argo_rate"].append(a_rate)
        #records["heuristic_rate"].append(h_rate)
        #records["performance"].append(perf)
      else:
        not_in_record.add(name)
    elif submit < contest_start:
      break
  else:
    page += 1
    continue
  break

if not_in_record: print("バチャ期間中に提出があったが、参加者に登録されていない：", not_in_record)

バチャ期間中に提出があったが、参加者に登録されていない： {'G4NP0N'}


In [10]:
df = pd.DataFrame(data = records)
df

,name,team,lang,score,submit,elapsed_time
0,r3yohei,バベルクライマーズ,Rust (1.42.0),47637101331,2022-10-20 18:59:44,4995 ms
1,jabee,a･Cyan,C++ (GCC 9.2.1),48643175751,2022-10-20 18:56:31,4949 ms
2,ponjuice,松竹梅,C++ (GCC 9.2.1),48154137543,2022-10-20 18:54:16,4958 ms
3,keroru,松竹梅,PyPy3 (7.3.0),48052667826,2022-10-20 18:48:47,4960 ms
4,ainem,運営,Rust (1.42.0),47945432845,2022-10-20 18:33:53,4985 ms
...,...,...,...,...,...,...
273,tnodino,ブレインバクハーズ,Python (3.8.2),3584631541,2022-10-13 20:43:42,4028 ms
274,keroru,松竹梅,PyPy3 (7.3.0),823090,2022-10-13 20:41:59,115 ms
275,Gex777,家買え,C++ (GCC 9.2.1),29415581618,2022-10-13 20:41:58,37 ms
276,gengar_094,ブレインバクハーズ,PyPy3 (7.3.0),823090,2022-10-13 19:58:48,178 ms


In [11]:
df.to_csv("submissions.csv")